In [2]:
#

In [42]:
%load_ext autoreload
%autoreload 2


from ff_energy.ffe.slurm import SlurmJobHandler
from pathlib import Path
import pandas as pd
import jax.numpy as jnp
import itertools as it
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import patchworklib as pw


from ff_energy.ffe.potential import (
    LJ,
    DE,
)

from ff_energy.ffe.ff import FF
from ff_energy.ffe.constants import FFEPATH, WORKINGDATA_PATH
from ff_energy.plotting.ffe_plots import plot_energy_MSE, plot_ff_fit

from ff_energy.plotting.plotting import set_style, patchwork_grid, save_fig


from ff_energy.plotting.fit_results import residuals_plot, residuals_from_keys
from ff_energy.logs.logging import hide_logs

from ff_energy.ffe.potential import LJ, LJ_bound, DE


hide_logs()


import scienceplots
set_style(no_latex=True)

# sns.set_style()
# plt.style.use(["science", "no-latex"])


from ff_energy.ffe.ff_fit import (
    load_ff,
    fit_func,
    fit_repeat,
)

from ff_energy.utils.ffe_utils import pickle_output, read_from_pickle, str2int, PKL_PATH
from ff_energy.utils.json_utils import load_json

from ff_energy.ffe.structure import atom_key_pairs


structure_data = {
    "dcm": "",
    "water_cluster": PKL_PATH / "20230823_water_clusters.pkl.pkl",
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
ions1_ci = pd.read_csv(WORKINGDATA_PATH / "ions1_ci.txt", sep=" ", header=None)
ions_ci = pd.read_csv(WORKINGDATA_PATH / "ions_ci.txt", sep=" ", header=None)
ion1_pol = pd.read_csv(WORKINGDATA_PATH / "ions1_pol.txt", sep=" ", header=None)
ions_pol = pd.read_csv(WORKINGDATA_PATH / "ions_pol.txt", sep=" ", header=None)
# WORKINGDATA_PATH / "ions1_pol.txt"
# WORKINGDATA_PATH / "ions_pol.txt"
ions_pol

,0,1
0,CLA_0_2,-18.061066
1,CLA_0_3,-4.643418
2,CLA_1_2,-9.571106
3,CLA_1_3,-14.511567
4,CLA_2_2,-12.563520
...,...,...
170,40_15_POT_16_1,-22.902988
171,40_1_POT_0_1,-28.582373
172,40_24_POT_26_1,-27.565464
173,40_29_POT_31_1,-36.502644


In [44]:
ions_ci.index = list(ions_ci[0])
ions_ci["KEY"] = list(ions_ci[0])
ions_ci["ELECci"] = list(ions_ci[1])
ions_ci.sort_index()

,0,1,KEY,ELECci
37_10_CLA_11_1,37_10_CLA_11_1,-151.301471,37_10_CLA_11_1,-151.301471
37_10_POT_9_1,37_10_POT_9_1,-111.442311,37_10_POT_9_1,-111.442311
37_11_CLA_13_1,37_11_CLA_13_1,-130.735784,37_11_CLA_13_1,-130.735784
37_11_POT_10_1,37_11_POT_10_1,-139.938094,37_11_POT_10_1,-139.938094
37_12_CLA_14_1,37_12_CLA_14_1,-163.983995,37_12_CLA_14_1,-163.983995
...,...,...,...,...
POT_5_1,POT_5_1,-227.918361,POT_5_1,-227.918361
POT_6_1,POT_6_1,-215.668214,POT_6_1,-215.668214
POT_7_1,POT_7_1,-245.735236,POT_7_1,-245.735236
POT_8_1,POT_8_1,-198.840149,POT_8_1,-198.840149


In [45]:
ions1_data = next(read_from_pickle(PKL_PATH / "ions_pbe0dz_pc.pkl"))
ions1_data

FileNotFoundError: [Errno 2] No such file or directory: '/home/boittier/Documents/phd/ff_energy/pickles/ions_pbe0dz_pc.pkl'

In [58]:
ions_data = next(read_from_pickle(PKL_PATH / "ions_ext_pbe0dz_pc.pkl"))
ions_data.data

,TOTAL,ELEC,VDW,KEY,M_ENERGY,n_monomers,C_ENERGY,P_ENERGY,P_intE,intE,C_ENERGY_kcalmol
0,24.23070,-85.88019,16.94441,37_33_POT_34_1,-1515.860920,13,-1515.972674,-18190.466947,-85.283983,-70.126190,-951277.400853
1,19.55507,-100.42969,20.12644,40_21_POT_23_1,-1592.230151,14,-1592.357754,-20699.150092,-99.223632,-80.070933,-999209.267558
2,17.02060,-109.44001,17.48012,40_14_CLA_14_1,-1452.779380,14,-1452.937435,-18886.330592,-124.654368,-99.179704,-911722.598974
3,18.46199,-102.74972,15.87134,37_2_CLA_1_1,-1376.419556,13,-1376.565542,-16517.217087,-114.465425,-91.606534,-863799.007176
4,20.01757,-90.99504,15.82786,37_36_POT_37_1,-1515.873093,13,-1516.002653,-18190.635741,-99.534006,-81.298711,-951296.212458
...,...,...,...,...,...,...,...,...,...,...,...
142,15.21813,-104.11307,16.26668,40_16_CLA_16_1,-1452.784550,14,-1452.942665,-18886.393761,-4.167002,-99.218126,-911725.881417
143,22.90285,-112.47656,19.92146,37_22_CLA_24_1,-1376.406502,13,-1376.561897,-16517.068478,NaN,-97.511080,-863796.720179
144,22.73310,-101.05692,14.85134,40_36_CLA_38_1,-1452.769618,14,-1452.914755,-18886.191067,0.505426,-91.074041,-911708.367500
145,19.29092,-102.93492,18.50733,40_35_POT_37_1,-1592.232052,14,-1592.375199,-20699.189787,-9.092990,-89.824927,-999220.214278


In [67]:
mdcm_ions_data = next(read_from_pickle(PKL_PATH / "ions_ext_pbe0dz_mdcm.pkl"))
# tip3 = mdcm_ions_data.data[["KEY", "ELEC"]]
# tip3
mdcm = mdcm_ions_data.data[["KEY", "ELEC"]]
mdcm["ELECm"] = mdcm["ELEC"]
mdcm

,KEY,ELEC,ELECm
0,37_33_POT_34_1,-89.34343,-89.34343
1,40_21_POT_23_1,-105.61965,-105.61965
2,40_14_CLA_14_1,-122.72154,-122.72154
3,37_2_CLA_1_1,-112.25992,-112.25992
4,37_36_POT_37_1,-98.26644,-98.26644
...,...,...,...
142,40_16_CLA_16_1,-117.79965,-117.79965
143,37_22_CLA_24_1,-125.31070,-125.31070
144,40_36_CLA_38_1,-112.96040,-112.96040
145,40_35_POT_37_1,-110.81841,-110.81841


In [69]:
tip3["ELECt"] = tip3["ELEC"]
tip3

,KEY,ELEC,ELECt
0,37_33_POT_34_1,-109.88936,-109.88936
1,40_21_POT_23_1,-133.58561,-133.58561
2,40_14_CLA_14_1,-139.06392,-139.06392
3,37_2_CLA_1_1,-131.81388,-131.81388
4,37_36_POT_37_1,-117.65454,-117.65454
...,...,...,...
142,40_16_CLA_16_1,-132.90271,-132.90271
143,37_22_CLA_24_1,-146.04534,-146.04534
144,40_36_CLA_38_1,-128.07845,-128.07845
145,40_35_POT_37_1,-133.54106,-133.54106


In [47]:
ions_data_keys = list(ions_data.data["KEY"])
ions_data.data.index = ions_data_keys
test_data = ions_data.data.dropna().copy()
test_data

,TOTAL,ELEC,VDW,KEY,M_ENERGY,n_monomers,C_ENERGY,P_ENERGY,P_intE,intE,C_ENERGY_kcalmol
37_33_POT_34_1,23.91634,-109.88936,16.94441,37_33_POT_34_1,-1515.860920,13,-1515.972674,-18190.466947,-85.283983,-70.126190,-951277.400853
40_21_POT_23_1,18.94132,-133.58561,20.12644,40_21_POT_23_1,-1592.230151,14,-1592.357754,-20699.150092,-99.223632,-80.070933,-999209.267558
40_14_CLA_14_1,16.62336,-139.06392,17.48012,40_14_CLA_14_1,-1452.779380,14,-1452.937435,-18886.330592,-124.654368,-99.179704,-911722.598974
37_2_CLA_1_1,18.42279,-131.81388,15.87134,37_2_CLA_1_1,-1376.419556,13,-1376.565542,-16517.217087,-114.465425,-91.606534,-863799.007176
37_36_POT_37_1,19.79936,-117.65454,15.82786,37_36_POT_37_1,-1515.873093,13,-1516.002653,-18190.635741,-99.534006,-81.298711,-951296.212458
...,...,...,...,...,...,...,...,...,...,...,...
40_6_CLA_6_1,21.02764,-138.67995,16.30033,40_6_CLA_6_1,-1452.766496,14,-1452.927202,-18886.160340,-2.733917,-100.843121,-911716.177949
40_24_CLA_24_1,20.26916,-160.73668,22.88985,40_24_CLA_24_1,-1452.765346,14,-1452.931950,-18886.150444,-126.092329,-104.544227,-911719.157239
40_16_CLA_16_1,15.03997,-132.90271,16.26668,40_16_CLA_16_1,-1452.784550,14,-1452.942665,-18886.393761,-4.167002,-99.218126,-911725.881417
40_36_CLA_38_1,22.50542,-128.07845,14.85134,40_36_CLA_38_1,-1452.769618,14,-1452.914755,-18886.191067,0.505426,-91.074041,-911708.367500


In [73]:
ions_pol["KEY"] = ions_pol[0]
ions_pol["pol"] = ions_pol[1]
ions_pol

,0,1,KEY,pol
0,CLA_0_2,-18.061066,CLA_0_2,-18.061066
1,CLA_0_3,-4.643418,CLA_0_3,-4.643418
2,CLA_1_2,-9.571106,CLA_1_2,-9.571106
3,CLA_1_3,-14.511567,CLA_1_3,-14.511567
4,CLA_2_2,-12.563520,CLA_2_2,-12.563520
...,...,...,...,...
170,40_15_POT_16_1,-22.902988,40_15_POT_16_1,-22.902988
171,40_1_POT_0_1,-28.582373,40_1_POT_0_1,-28.582373
172,40_24_POT_26_1,-27.565464,40_24_POT_26_1,-27.565464
173,40_29_POT_31_1,-36.502644,40_29_POT_31_1,-36.502644


In [82]:
combined_df = pd.merge(ions_data.data, ions_ci, on="KEY").dropna()

combined_df = pd.merge(combined_df, ions_pol[["KEY", "pol"]], on="KEY").dropna()

combined_df = pd.merge(combined_df, tip3[["KEY", "ELECt"]], on="KEY").dropna()

combined_df = pd.merge(combined_df, mdcm[["KEY", "ELECm"]], on="KEY").dropna()

combined_df["ELECp"] = combined_df["ELEC"]

combined_df["ELECpol"] = combined_df["pol"] + combined_df["ELECci"]
combined_df["ELECppol"] = combined_df["pol"] + combined_df["ELECp"]
combined_df["ELECmpol"] = combined_df["pol"] + combined_df["ELECm"]
combined_df["ELECtpol"] = combined_df["pol"] + combined_df["ELECt"]

combined_df["ion"] = combined_df["KEY"].apply(lambda x: x.split("_")[2])

# combined_df[combined_df["ion"] == "POT"]
# combined_df[combined_df["ion"] == "CLA"]
combined_df.index = combined_df["KEY"]
combined_df

,TOTAL,ELEC,VDW,KEY,M_ENERGY,n_monomers,C_ENERGY,P_ENERGY,P_intE,intE,...,ELECci,pol,ELECt,ELECm,ELECp,ELECpol,ELECppol,ELECmpol,ELECtpol,ion
KEY,,,,,,,,,,,,,,,,,,,,,
37_33_POT_34_1,24.23070,-85.88019,16.94441,37_33_POT_34_1,-1515.860920,13,-1515.972674,-18190.466947,-85.283983,-70.126190,...,-119.617917,-25.472746,-109.88936,-89.34343,-85.88019,-145.090664,-111.352936,-114.816176,-135.362106,POT
40_21_POT_23_1,19.55507,-100.42969,20.12644,40_21_POT_23_1,-1592.230151,14,-1592.357754,-20699.150092,-99.223632,-80.070933,...,-152.874191,-27.989344,-133.58561,-105.61965,-100.42969,-180.863535,-128.419034,-133.608994,-161.574954,POT
40_14_CLA_14_1,17.02060,-109.44001,17.48012,40_14_CLA_14_1,-1452.779380,14,-1452.937435,-18886.330592,-124.654368,-99.179704,...,-177.110604,-0.130172,-139.06392,-122.72154,-109.44001,-177.240776,-109.570182,-122.851712,-139.194092,CLA
37_2_CLA_1_1,18.46199,-102.74972,15.87134,37_2_CLA_1_1,-1376.419556,13,-1376.565542,-16517.217087,-114.465425,-91.606534,...,-161.670623,-1.780261,-131.81388,-112.25992,-102.74972,-163.450884,-104.529981,-114.040181,-133.594141,CLA
37_36_POT_37_1,20.01757,-90.99504,15.82786,37_36_POT_37_1,-1515.873093,13,-1516.002653,-18190.635741,-99.534006,-81.298711,...,-132.801863,-28.071169,-117.65454,-98.26644,-90.99504,-160.873032,-119.066209,-126.337609,-145.725709,POT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40_6_CLA_6_1,22.06840,-107.49668,16.30033,40_6_CLA_6_1,-1452.766496,14,-1452.927202,-18886.160340,-2.733917,-100.843121,...,-173.941166,-5.735368,-138.67995,-121.13256,-107.49668,-179.676535,-113.232048,-126.867928,-144.415318,CLA
40_24_CLA_24_1,21.42775,-121.12485,22.88985,40_24_CLA_24_1,-1452.765346,14,-1452.931950,-18886.150444,-126.092329,-104.544227,...,-195.859405,-4.657902,-160.73668,-135.95314,-121.12485,-200.517308,-125.782752,-140.611042,-165.394582,CLA
40_16_CLA_16_1,15.21813,-104.11307,16.26668,40_16_CLA_16_1,-1452.784550,14,-1452.942665,-18886.393761,-4.167002,-99.218126,...,-170.122389,-3.238031,-132.90271,-117.79965,-104.11307,-173.360420,-107.351101,-121.037681,-136.140741,CLA


In [83]:
pickle_output(combined_df, PKL_PATH / "20230919_ions_ext.pkl")

In [84]:
combined_df.keys()

0 ('C', 'C')
1 ('C', 'CG331')
2 ('C', 'CL')
3 ('C', 'CLA')
4 ('C', 'H')
5 ('C', 'HGA3')
6 ('C', 'HGP1')
7 ('C', 'HT')
8 ('C', 'OG311')
9 ('C', 'OT')
10 ('C', 'POT')
11 ('CG331', 'CG331')
12 ('CG331', 'CL')
13 ('CG331', 'CLA')
14 ('CG331', 'H')
15 ('CG331', 'HGA3')
16 ('CG331', 'HGP1')
17 ('CG331', 'HT')
18 ('CG331', 'OG311')
19 ('CG331', 'OT')
20 ('CG331', 'POT')
21 ('CL', 'CL')
22 ('CL', 'CLA')
23 ('CL', 'H')
24 ('CL', 'HGA3')
25 ('CL', 'HGP1')
26 ('CL', 'HT')
27 ('CL', 'OG311')
28 ('CL', 'OT')
29 ('CL', 'POT')
30 ('CLA', 'CLA')
31 ('CLA', 'H')
32 ('CLA', 'HGA3')
33 ('CLA', 'HGP1')
34 ('CLA', 'HT')
35 ('CLA', 'OG311')
36 ('CLA', 'OT')
37 ('CLA', 'POT')
38 ('H', 'H')
39 ('H', 'HGA3')
40 ('H', 'HGP1')
41 ('H', 'HT')
42 ('H', 'OG311')
43 ('H', 'OT')
44 ('H', 'POT')
45 ('HGA3', 'HGA3')
46 ('HGA3', 'HGP1')
47 ('HGA3', 'HT')
48 ('HGA3', 'OG311')
49 ('HGA3', 'OT')
50 ('HGA3', 'POT')
51 ('HGP1', 'HGP1')
52 ('HGP1', 'HT')
53 ('HGP1', 'OG311')
54 ('HGP1', 'OT')
55 ('HGP1', 'POT')
56 ('HT', 'HT'

Index([           'TOTAL',             'ELEC',              'VDW',
                    'KEY',         'M_ENERGY',       'n_monomers',
               'C_ENERGY',         'P_ENERGY',           'P_intE',
                   'intE', 'C_ENERGY_kcalmol',                  0,
                        1,           'ELECci',              'pol',
                  'ELECt',            'ELECm',            'ELECp',
                'ELECpol',         'ELECppol',         'ELECmpol',
               'ELECtpol',              'ion'],
      dtype='object')

In [49]:
ions_data_keys

['37_33_POT_34_1',
 '40_21_POT_23_1',
 '40_14_CLA_14_1',
 '37_2_CLA_1_1',
 '37_36_POT_37_1',
 '37_27_POT_28_1',
 '40_13_CLA_13_1',
 '37_26_CLA_28_1',
 '37_4_POT_3_1',
 '37_34_CLA_36_1',
 '37_17_CLA_19_1',
 '37_27_CLA_29_1',
 '37_35_POT_36_1',
 '40_7_POT_7_1',
 '40_33_CLA_34_1',
 '37_9_POT_8_1',
 '37_32_CLA_34_1',
 '37_14_POT_13_1',
 '40_32_CLA_32_1',
 '40_11_CLA_11_1',
 '37_21_POT_22_1',
 '40_33_POT_35_1',
 '40_8_CLA_8_1',
 '40_3_POT_2_1',
 '40_28_POT_30_1',
 '37_16_CLA_18_1',
 '37_16_POT_16_1',
 '40_9_POT_9_1',
 '40_5_CLA_5_1',
 '37_3_CLA_2_1',
 '40_23_CLA_23_1',
 '40_30_POT_32_1',
 '40_29_POT_31_1',
 '40_19_CLA_19_1',
 '40_5_POT_4_1',
 '40_11_POT_11_1',
 '40_17_POT_18_1',
 '37_34_POT_35_1',
 '37_1_POT_0_1',
 '40_8_POT_8_1',
 '40_10_POT_10_1',
 '37_21_CLA_23_1',
 '37_24_POT_25_1',
 '40_27_CLA_27_1',
 '40_22_POT_24_1',
 '37_17_POT_17_1',
 '40_19_POT_20_1',
 '40_31_CLA_31_1',
 '37_13_CLA_15_1',
 '40_32_POT_34_1',
 '40_1_POT_0_1',
 '40_16_POT_17_1',
 '37_4_CLA_3_1',
 '40_20_CLA_20_1',
 '